In [2]:
%env THEANO_FLAGS=device=gpu0
import sys
sys.path.append("..")

env: THEANO_FLAGS=device=gpu0


In [3]:
from utils.southpark import get_conversations
#southpark stuff
conversations_southpark = get_conversations("/srv/hd7/jheuristic/cartman/sp.json")


In [4]:
from utils.preprocessor import Preprocessor
sp_tokens = Preprocessor.from_conversations(conversations_southpark,verbose=True).tokens[3:]


100%|██████████| 7285/7285 [00:03<00:00, 2160.68it/s]

23281 out of 23281 tokens, coverage=1.00000)


In [5]:
from tqdm import tqdm
with open("/srv/hd7/jheuristic/cartman/OpenSubtitles2016.en-ru.en") as f:
    opensub_lines = [['opensub',line] for line in tqdm(f)]
    
batch_size = 100
opensub_conv_chunks = [opensub_lines[i:i+batch_size] 
                       for i in range(0,len(opensub_lines) - batch_size +1,batch_size)]


18713837it [00:15, 1205766.09it/s]


In [ ]:
os_tokens = Preprocessor.from_conversations(opensub_conv_chunks,max_tokens=50000,verbose=True).tokens[3:]

In [ ]:
preproc = Preprocessor(list(set(os_tokens+sp_tokens)))

In [10]:
import pickle
with open("/srv/hd7/jheuristic/Downloads/opensub_en_tokens.pcl",'w') as fout:
    pickle.dump(preproc.tokens,fout)

In [6]:
from utils.preprocessor import Preprocessor
import pickle
with open("/srv/hd7/jheuristic/Downloads/opensub_en_tokens.pcl") as fin:
    preproc = Preprocessor(pickle.load(fin))

../utils/preprocessor.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  token_to_ix = {t:i for i,t in enumerate(tokens)}
../utils/preprocessor.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  token_to_ix = defaultdict(lambda:token_to_ix["UNK"],token_to_ix)


# model

In [7]:
from models.twoline_model import ConversationModel
model = ConversationModel(preproc,bottleneck_size=2048,emb_size=512)

Using gpu device 0: Tesla K40m (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 4004.)
/home/jheuristic/thenv/local/lib/python2.7/site-packages/agentnet/agent/recurrence.py:228: UserWarning: You are giving Recurrence an input sequence of undefined length (None).
Make sure it is always above <unspecified>(n_steps) you specified for recurrence
  "Make sure it is always above {}(n_steps) you specified for recurrence".format(n_steps or "<unspecified>"))
/home/jheuristic/thenv/local/lib/python2.7/site-packages/agentnet/agent/recurrence.py:453: UserWarning: Warning: recurrent loop without unroll_scan got nonempty random state updates list. That happened because there is some source of randomness (e.g. dropout) inside recurrent step graph. To compile such graph, one must either call .get_automatic_updates() right after .get_output and pass these updates to a function, or use no_defalt_updates=True when compiling theano.function.
  warn("Warning: recurrent loop without unroll_sc

# training

In [8]:
conversations = conversations_southpark + opensub_conv_chunks

In [ ]:
max_len=30
batch_size=25
n_epochs=100

In [ ]:
from utils.generate import get_phrase_pairs,iterate_minibatches

convs_ix = list(preproc.preprocess_conversations(conversations,verbose=True,max_len=max_len))

prev_phrases,reference_answers = get_phrase_pairs(convs_ix,
                                                  speaker_filter = lambda s1,s2: True).swapaxes(0,1).astype('int32')




  0%|          | 0/194423 [00:00<?, ?it/s]../utils/preprocessor.py:75: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  tokens = [self.token_to_ix[t] for t in self.tokenizer.tokenize(self.preprocess_phrase(phrase))]
  5%|▍         | 8940/194423 [00:09<09:30, 325.19it/s]

In [ ]:
epoch_counter = 1
ce = []

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for _ in range(n_epochs):
    for b_x,b_y in iterate_minibatches([prev_phrases,reference_answers],batch_size,shuffle=True):
        ce.append(model.train_fun(b_x,b_y))
        
        epoch_counter +=1
        
        if epoch_counter %25==0:
            print epoch_counter,'iterations...'
    print "beginning new loop..."
    plt.plot(ce)
    plt.show()
    
    ph =preproc.ix_to_phrase(b_x[0])
    print 'A:', ph
    print 'B:', model.reply(ph)
    print 'B true:',preproc.ix_to_phrase(b_y[0])
        

In [ ]:
from agentnet.utils.persistence import save
save(model.decoder_training_rec,"/srv/hd7/jheuristic/agentnet_snapshots/opensub_simple_cm_1024n_cartman_epoch%i.pcl"%epoch_counter)